<a href="https://colab.research.google.com/github/Chatura-17/Re_ID_Assignment/blob/main/Re_Identification_Ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python-headless numpy matplotlib



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [3]:
from ultralytics import YOLO

model_path = '/content/drive/MyDrive/Internship_Assignment/best.pt'

model = YOLO(model_path)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [16]:
import cv2
import numpy as np
import os
import pandas as pd

video_path = '/content/drive/MyDrive/Internship_Assignment/15sec_input_720p.mp4'
output_video_path = '/content/drive/MyDrive/Internship_Assignment/output_annotated.avi'
output_csv_path = '/content/drive/MyDrive/Internship_Assignment/player_tracking.csv'

DIST_THRESH = 50
HIST_THRESH = 0.5
FRAME_RATE = 30
INITIAL_FRAMES = 45

next_id = 0
players = {}

def get_color_histogram(crop):
    hist = cv2.calcHist([crop], [0, 1, 2], None, [8, 8, 8],
                        [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def cosine_similarity(h1, h2):
    if np.linalg.norm(h1) == 0 or np.linalg.norm(h2) == 0:
        return 0
    return np.dot(h1, h2) / (np.linalg.norm(h1) * np.linalg.norm(h2))

def detect_players(frame):
    results = model.predict(frame, conf=0.3)[0]
    boxes = []
    centers = []
    crops = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        cls = int(box.cls[0])
        if cls != 0:
            continue
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)
        boxes.append((x1, y1, x2, y2))
        centers.append((cx, cy))
        crops.append(frame[y1:y2, x1:x2])
    return boxes, centers, crops

def assign_id(center, hist, frame_no):
    global next_id
    best_match = None
    best_score = 0
    for pid, data in players.items():
        dist = np.linalg.norm(np.array(center) - np.array(data['center']))
        if dist < DIST_THRESH:
            sim = cosine_similarity(hist, data['hist'])
            if sim > HIST_THRESH and sim > best_score:
                best_match = pid
                best_score = sim
    if best_match is not None:
        players[best_match]['center'] = center
        players[best_match]['hist'] = hist
        players[best_match]['last_seen'] = frame_no
        return best_match
    else:
        players[next_id] = {'center': center, 'hist': hist, 'last_seen': frame_no}
        next_id += 1
        return next_id - 1

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(3))
height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

csv_log = []

frame_no = 0
print("Processing video...")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    boxes, centers, crops = detect_players(frame)
    assigned_ids = []

    for i in range(len(centers)):
        center = centers[i]
        crop = crops[i]
        hist = get_color_histogram(crop)
        pid = assign_id(center, hist, frame_no)
        assigned_ids.append(pid)

        x1, y1, x2, y2 = boxes[i]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {pid}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        csv_log.append({
            'frame': frame_no,
            'player_id': pid,
            'x_center': center[0],
            'y_center': center[1]
        })

    out.write(frame)
    frame_no += 1

cap.release()
out.release()
print("Video saved:", output_video_path)

# Save CSV
df = pd.DataFrame(csv_log)
df.to_csv(output_csv_path, index=False)
print("CSV saved:", output_csv_path)


Processing video...

0: 384x640 1 ball, 16 players, 2 referees, 5324.7ms
Speed: 3.2ms preprocess, 5324.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 6561.5ms
Speed: 3.8ms preprocess, 6561.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2529.3ms
Speed: 3.3ms preprocess, 2529.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2445.7ms
Speed: 3.2ms preprocess, 2445.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2488.6ms
Speed: 3.1ms preprocess, 2488.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 players, 2 referees, 3267.7ms
Speed: 2.9ms preprocess, 3267.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 3699.3ms
Speed: 3.6ms preprocess, 3699.3ms infer